# MRKL

This notebook showcases using an agent to replicate the MRKL chain.

This uses the example Chinook database.
To set it up follow the instructions on https://database.guide/2-sample-databases-sqlite/, placing the `.db` file in a notebooks folder at the root of this repository.

In [1]:
from langchain import LLMMathChain, OpenAI, SerpAPIWrapper, SQLDatabase, SQLDatabaseChain
from langchain.agents import initialize_agent, Tool

In [3]:
llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
db = SQLDatabase.from_uri("sqlite:///../../../../notebooks/Chinook.db")
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
    Tool(
        name="FooBar DB",
        func=db_chain.run,
        description="useful for when you need to answer questions about FooBar. Input should be in the form of a question containing full context"
    )
]

In [4]:
mrkl = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [5]:
mrkl.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")



> Entering new AgentExecutor chain...
 I need to find out who Olivia Wilde's boyfriend is and then calculate his age raised to the 0.23 power.
Action: Search
Action Input: "Who is Olivia Wilde's boyfriend?"
Observation: Harry Styles
Thought: I need to find out Harry Styles' age
Action: Search
Action Input: "How old is Harry Styles?"
Observation: 28 years
Thought: I need to calculate 28 raised to the 0.23 power
Action: Calculator
Action Input: 28^0.23

> Entering new LLMMathChain chain...
28^0.23
```python
import math
print(math.pow(28, 0.23))
```

Answer: 2.1520202182226886

> Finished LLMMathChain chain.

Observation: Answer: 2.1520202182226886

Thought: I now know the final answer
Final Answer: Harry Styles is 28 years old and his age raised to the 0.23 power is 2.1520202182226886.
> Finished AgentExecutor chain.


'Harry Styles is 28 years old and his age raised to the 0.23 power is 2.1520202182226886.'

In [6]:
mrkl.run("What is the full name of the artist who recently released an album called 'The Storm Before the Calm' and are they in the FooBar database? If so, what albums of theirs are in the FooBar database?")



> Entering new AgentExecutor chain...
 I need to find out the artist's full name and then search the FooBar database for their albums.
Action: Search
Action Input: "The Storm Before the Calm" artist
Observation: Alanis Morissette - the storm before the calm - Amazon.com Music.
Thought: I now need to search the FooBar database for Alanis Morissette's albums.
Action: FooBar DB
Action Input: What albums of Alanis Morissette are in the FooBar database?

> Entering new SQLDatabaseChain chain...
What albums of Alanis Morissette are in the FooBar database? 
SQLQuery: SELECT Title FROM Album WHERE ArtistId IN (SELECT ArtistId FROM Artist WHERE Name = 'Alanis Morissette');
SQLResult: [('Jagged Little Pill',)]
Answer: The album 'Jagged Little Pill' by Alanis Morissette is in the FooBar database.
> Finished SQLDatabaseChain chain.

Observation:  The album 'Jagged Little Pill' by Alanis Morissette is in the FooBar database.
Thought: I now know the final answer.
Final Answer: Alanis Morissette is

"Alanis Morissette is the artist who recently released an album called 'The Storm Before the Calm' and the album 'Jagged Little Pill' by Alanis Morissette is in the FooBar database."